In [100]:
import os
import openai
import pandas as pd
import pdfplumber
import json
import numpy as np

import config

In [18]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [2]:
#%env OPENAI_API_KEY= API_KEY

In [4]:
import openai
from openai import OpenAI
client = OpenAI()

In [324]:
def rename_columns(df):
    import pandas as pd
    
    # Rename columns in df to match df_truth
    df = df.rename(columns={
        "Last Name of Main Author and Year": "References",
        "Full Population Sample Size": "Sample size",
        "Main Race": "Main race",
        "Intervention": "Interventions",
        "Percent of Study Population that is Female (%)": "Female (%)",
        "Percent of Intervention Population that is Female (%)": "Female (%)",
        "Mean HAMA Score": "Mean HAMA",
        "Mean Population Age (Year)": "Mean age (Year)",
        "Attrition Rate (%)": "Attrition rate (%)",
        "Full Sponsor Name": "Sponsor",
        "Follow-up Time (Weeks)": "Follow-up time (weeks)",
        "Diagnostic Criteria": "Diagnosis criteria"
    })
    return df

# Training

In [6]:
def get_file_names(directory):
    """Gets a list of file names in the specified directory.

    Args:
        directory (str): The path to the directory.

    Returns:
        list: A list of file names.
    """

    file_names = []
    for entry in os.scandir(directory):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

directory_path = "../Desktop/Training"
file_names = get_file_names(directory_path)
print(file_names)

['stein2017.pdf', 'bidzan2012.pdf', 'coric2010.pdf', 'stein2014.pdf', 'feltner2003.pdf', 'koponen2007.pdf', 'montgomery2008.pdf', 'pollock2008b.pdf', 'bandelow2010.pdf', 'mezhebovsky2013.pdf']


# Training Set - Text only

In [316]:
# Function to extract text and tables
def extract_pdf_text(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text += page.extract_text() + "\n"
    return text


def query_gpt4_text(text):
    structured_prompt = (
        f"Clinical Trial Report Analysis:\n\n"
        f"Extracted Text:\n{text}\n\n"
        f"This is a clinical trial report. For EACH intervention in the trial (including placebo), please extract the following characteristics and format the response as valid JSON using this exact example structure:\n\n"
        f"Example format:\n"
        f"{{\n"
        f'    "Last Name of Main Author and Year": "Doe et al., 2021",\n'
        f'    "Full Population Sample Size": "451",\n'
        f'    "Intervention": "Duloxetine: 50 mg/day",\n'
        f'    "Main Race": "White",\n'
        f'    "Percent of Intervention Population that is Female (%)": "61.5",\n'
        f'    "Mean HAMA Score": "24.5",\n'
        f'    "Mean Population Age (Year)": "43.2",\n'
        f'    "Attrition Rate (%)": "30.2",\n'
        f'    "Full Sponsor Name": "ABC Pharmaceuticals",\n'
        f'    "Follow-up Time (Weeks)": "10",\n'
        f'    "Diagnostic Criteria": "DSM-IV"\n'
        f"}}\n\n"
        f"'Full Population Sample Size' should refer to the TOTAL population enrolled in the study, across all interventions and groups, not just the population size for the specific intervention.\n"
        f"'Intervention' should be in mg/day, not any other unit of measurement.\n"
        f"Make sure each JSON object follows this format exactly. For any missing or unavailable data, input 'NA'."
    )
    response = completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a data extraction assistant extracting data from clinical trial reports."},
                  {"role": "user", "content": structured_prompt}],
    )
    return response

## Stein 2017

In [ ]:
file_path = "../Desktop/Training/stein2017.pdf"

# Extract content from PDF
stein_2017_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_stein_2017 = query_gpt4_text(stein_2017_text)

In [12]:
print("Extracted Data:", structured_data_stein_2017.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Stein et al., 2017",
    "Full Population Sample Size": "412",
    "Intervention": "Agomelatine: 10 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "67.7",
    "Mean HAMA Score": "28.6",
    "Mean Population Age (Year)": "43.9",
    "Attrition Rate (%)": "13.7",
    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",
    "Follow-up Time (Weeks)": "12",
    "Diagnostic Criteria": "DSM-IV-TR"
}
{
    "Last Name of Main Author and Year": "Stein et al., 2017",
    "Full Population Sample Size": "412",
    "Intervention": "Agomelatine: 25 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "67.7",
    "Mean HAMA Score": "29.0",
    "Mean Population Age (Year)": "44.1",
    "Attrition Rate (%)": "9.4",
    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",
    "Follow-up Time (Weeks)": "12",
    "Diagn

In [16]:
clean_output(structured_data_stein_2017)

Error parsing JSON: Extra data: line 14 column 1 (char 504)


""


In [70]:
structured_data_stein_2017.choices[0].message.content

'```json\n{\n    "Last Name of Main Author and Year": "Stein et al., 2017",\n    "Full Population Sample Size": "412",\n    "Intervention": "Agomelatine: 10 mg/day",\n    "Main Race": "NA",\n    "Percent of Study Population that is Female (%)": "67.7",\n    "Mean HAMA Score": "28.6",\n    "Mean Population Age (Year)": "43.9",\n    "Attrition Rate (%)": "13.7",\n    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",\n    "Follow-up Time (Weeks)": "12",\n    "Diagnostic Criteria": "DSM-IV-TR"\n}\n{\n    "Last Name of Main Author and Year": "Stein et al., 2017",\n    "Full Population Sample Size": "412",\n    "Intervention": "Agomelatine: 25 mg/day",\n    "Main Race": "NA",\n    "Percent of Study Population that is Female (%)": "67.7",\n    "Mean HAMA Score": "29.0",\n    "Mean Population Age (Year)": "44.1",\n    "Attrition Rate (%)": "9.4",\n    "Full Sponsor Name": "Institut de Recherches Internationales Servier (IRIS)",\n    "Follow-up Time (Weeks)": "12",\n

In [80]:
# Step 1: Clean the text if wrapped in code block markers
text = structured_data_stein_2017.choices[0].message.content
text = text.replace("```json\n{\n", "```json\n[\n    {\n")
text = text.replace("\n}\n```", "\n    }\n]\n```")
text = text.replace("}\n{","},\n{")

if text.startswith("```json"):
    text = text[len("```json"):].strip()
if text.endswith("```"):
    text = text[:-len("```")].strip()

# Step 2: Parse the JSON list directly
try:
    data = json.loads(text)  # Parse the JSON list directly
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    data = []

# Step 3: Convert the list of dictionaries into a pandas DataFrame
stein_2017_df = pd.DataFrame(data)


In [88]:
stein_2017_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Stein et al., 2017",412,Agomelatine: 10 mg/day,NA,67.7,28.6,43.9,13.7,Institut de Recherches Internationales Servier...,12,DSM-IV-TR
1,"Stein et al., 2017",412,Agomelatine: 25 mg/day,NA,67.7,29.0,44.1,9.4,Institut de Recherches Internationales Servier...,12,DSM-IV-TR
2,"Stein et al., 2017",412,Placebo,NA,67.7,28.8,44.1,21.1,Institut de Recherches Internationales Servier...,12,DSM-IV-TR


In [89]:
stein_2017_df = rename_columns(stein_2017_df)

In [155]:
def preprocess_numerical_columns(df, numerical_cols):
    """
    Converts specified numerical columns to numeric types in a DataFrame.
    Keeps 'NA' as a string and does not convert it to NaN.
    """
    for col in numerical_cols:
        if col in df.columns:
            # Preserve 'NA' and convert the rest to numeric
            df[col] = df[col].apply(lambda x: x if str(x).strip().lower() == "na" else pd.to_numeric(x, errors="coerce"))
    return df

# Update the numerical columns list
numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]


In [91]:
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)
stein_2017_df = preprocess_numerical_columns(stein_2017_df, numerical_cols)

In [285]:
def preprocess_interventions_and_diag(df):
    """
    Normalizes the 'Interventions' column in the DataFrame for consistent filtering.
    Replaces all dashes (e.g., en dash, em dash) with a standard hyphen.
    """
    df = df.copy()
    df["Interventions"] = (
        df["Interventions"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
        .str.replace(r":(?=\d)", ": ", regex=True)
    )
    df["Diagnosis criteria"] = (
    df["Diagnosis criteria"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
    )
    return df

In [93]:
# Normalize the 'Interventions' column in both DataFrames
df_truth = preprocess_interventions_and_diag(df_truth)
stein_2017_df = preprocess_interventions_and_diag(stein_2017_df)

In [164]:
# def testing(df_truth, df):
#     grouped_truth = df_truth.groupby("References")
#     grouped_pred = df.groupby("References")
    
#     results = []
    
#     for reference in grouped_truth.groups:
#         if reference in grouped_pred.groups:
#             # Get groups
#             truth_group = grouped_truth.get_group(reference)
#             pred_group = grouped_pred.get_group(reference)
    
#             # Normalize and find common interventions
#             truth_interventions = set(truth_group["Interventions"].tolist())
#             pred_interventions = set(pred_group["Interventions"].tolist())
#             common_interventions = truth_interventions & pred_interventions  # Only common interventions
    
#             for intervention in common_interventions:
#                 # Filter for the specific intervention
#                 truth_row = truth_group[truth_group["Interventions"] == intervention]
#                 pred_row = pred_group[pred_group["Interventions"] == intervention]
    
#                 if not truth_row.empty and not pred_row.empty:
#                     truth_row = truth_row.iloc[0]
#                     pred_row = pred_row.iloc[0]
    
#                     # Categorical columns with exact matching
#                     categorical_cols = ["Main race"]
#                     categorical_match = {
#                         col: truth_row[col].strip().lower() == pred_row[col].strip().lower()
#                         for col in categorical_cols
#                     }
    
#                     # Special handling for "Diagnosis criteria"
#                     diagnosis_criteria_truth = truth_row["Diagnosis criteria"].strip().lower()
#                     diagnosis_criteria_pred = pred_row["Diagnosis criteria"].strip().lower()
#                     categorical_match["Diagnosis criteria"] = diagnosis_criteria_truth in diagnosis_criteria_pred

#                     # Special handling for "Sponsor"
#                     sponsor_truth = truth_row["Sponsor"].strip().lower()
#                     sponsor_pred = pred_row["Sponsor"].strip().lower()
#                     categorical_match["Sponsor"] = sponsor_truth in sponsor_pred

#                     # Numerical columns
#                     numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]
#                     numerical_match = {
#                         col: np.isclose(truth_row[col], pred_row[col], atol=.5, equal_nan=True)
#                         for col in numerical_cols
#                     }
    
#                     # Collect results
#                     results.append({
#                         "References": reference,
#                         "Interventions": intervention,
#                         **categorical_match,
#                         **numerical_match,
#                     })
#     results_df = pd.DataFrame(results)
#     return results_df

In [220]:
def testing(df_truth, df):
    grouped_truth = df_truth.groupby("References")
    grouped_pred = df.groupby("References")
    
    results = []
    
    for reference in grouped_truth.groups:
        if reference in grouped_pred.groups:
            # Get groups
            truth_group = grouped_truth.get_group(reference)
            pred_group = grouped_pred.get_group(reference)
    
            # Normalize and find common interventions
            truth_interventions = set(truth_group["Interventions"].tolist())
            pred_interventions = set(pred_group["Interventions"].tolist())
            common_interventions = truth_interventions & pred_interventions  # Only common interventions
    
            for intervention in common_interventions:
                # Filter for the specific intervention
                truth_row = truth_group[truth_group["Interventions"] == intervention]
                pred_row = pred_group[pred_group["Interventions"] == intervention]
    
                if not truth_row.empty and not pred_row.empty:
                    truth_row = truth_row.iloc[0]
                    pred_row = pred_row.iloc[0]
    
                    # Categorical columns
                    categorical_cols = ["Main race"]
                    categorical_match = {
                        col: truth_row[col].strip().lower() == pred_row[col].strip().lower()
                        for col in categorical_cols
                    }
    
                    # Special handling for "Diagnosis criteria"
                    diagnosis_criteria_truth = truth_row["Diagnosis criteria"].strip().lower()
                    diagnosis_criteria_pred = pred_row["Diagnosis criteria"].strip().lower()
                    categorical_match["Diagnosis criteria"] = diagnosis_criteria_truth in diagnosis_criteria_pred

                    # Special handling for "Sponsor"
                    sponsor_truth = truth_row["Sponsor"].strip().lower()
                    sponsor_pred = pred_row["Sponsor"].strip().lower()
                    categorical_match["Sponsor"] = sponsor_truth in sponsor_pred

                    # Numerical columns
                    numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]
                    numerical_match = {}
                    for col in numerical_cols:
                        truth_val = truth_row[col]
                        pred_val = pred_row[col]
                        truth_is_na = str(truth_val).strip().lower() == "na"
                        pred_is_na = str(pred_val).strip().lower() == "na"

                        # Handle different scenarios for NA
                        if truth_is_na and pred_is_na:
                            numerical_match[col] = True  # Both are NA, match is True
                        elif truth_is_na or pred_is_na:
                            numerical_match[col] = False  # Only one is NA, match is False
                        else:
                            # Perform numerical comparison
                            numerical_match[col] = np.isclose(
                                float(truth_val), float(pred_val), atol=0.5, equal_nan=True
                            )
    
                    # Collect results
                    results.append({
                        "References": reference,
                        "Interventions": intervention,
                        **categorical_match,
                        **numerical_match,
                    })
    results_df = pd.DataFrame(results)
    return results_df

In [119]:
stein_2017_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Stein et al., 2017",412,agomelatine: 10 mg/day,NA,67.7,28.6,43.9,13.7,Institut de Recherches Internationales Servier...,12,dsm-iv-tr
1,"Stein et al., 2017",412,agomelatine: 25 mg/day,NA,67.7,29.0,44.1,9.4,Institut de Recherches Internationales Servier...,12,dsm-iv-tr
2,"Stein et al., 2017",412,placebo,NA,67.7,28.8,44.1,21.1,Institut de Recherches Internationales Servier...,12,dsm-iv-tr


In [120]:
df_truth[df_truth["References"] == "Stein et al., 2017"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
20,"Stein et al., 2017",412,agomelatine: 10 mg/day,NA,67.9,28.6,43.6,13.7,Servier,12.0,dsm-iv
21,"Stein et al., 2017",412,agomelatine: 25 mg/day,NA,71.9,29.0,44.1,9.4,Servier,12.0,dsm-iv
22,"Stein et al., 2017",412,placebo,NA,63.4,28.8,44.1,21.1,Servier,12.0,dsm-iv


In [147]:
stein2017_results = testing(df_truth, stein_2017_df)

In [148]:
stein2017_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2017",agomelatine: 10 mg/day,True,True,True,True,True,True,True,True,True
1,"Stein et al., 2017",placebo,True,True,True,True,False,True,True,True,True
2,"Stein et al., 2017",agomelatine: 25 mg/day,True,True,True,True,False,True,True,True,True


In [149]:
def ensure_all_truth_interventions_present(results_df, df_truth):
    """
    Ensures all interventions in `df_truth` are present in `results_df`, grouped by reference.
    If an intervention is missing, adds a row with False for all metrics.
    """
    # Normalize `Interventions` in `df_truth` to match `results_df`
    df_truth = preprocess_interventions_and_diag(df_truth)
    df_truth = df_truth[df_truth["References"].isin(results_df["References"])]

    # Group by reference in both dataframes
    grouped_truth = df_truth.groupby("References")
    grouped_results = results_df.groupby("References")

    # Initialize a list to store missing rows
    missing_rows = []

    # Iterate through each reference in `df_truth`
    for reference, truth_group in grouped_truth:
        # Get the interventions for the current reference in `df_truth`
        truth_interventions = set(truth_group["Interventions"].tolist())

        # Get the interventions for the same reference in `results_df`, if it exists
        if reference in grouped_results.groups:
            results_group = grouped_results.get_group(reference)
            results_interventions = set(results_group["Interventions"].tolist())
        else:
            results_interventions = set()

        # Find missing interventions for this reference
        missing_interventions = truth_interventions - results_interventions

        # Create rows for missing interventions
        for intervention in missing_interventions:
            missing_rows.append({
                "References": reference,
                "Interventions": intervention,
                **{col: False for col in results_df.columns if col not in ["References", "Interventions"]}
            })

    # Add missing rows to results_df
    if missing_rows:
        results_df = pd.concat([results_df, pd.DataFrame(missing_rows)], ignore_index=True)

    return results_df

In [127]:
stein2017_results_final = ensure_all_truth_interventions_present(stein2017_results,df_truth)

In [128]:
stein2017_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2017",agomelatine: 10 mg/day,True,True,True,True,True,True,True,True,True
1,"Stein et al., 2017",placebo,True,True,True,True,False,True,True,True,True
2,"Stein et al., 2017",agomelatine: 25 mg/day,True,True,True,True,False,True,True,True,True


In [129]:
file_path = os.path.join("data_text", "stein2017data.csv")  
stein_2017_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "stein2017results.csv")  
stein2017_results_final.to_csv(file_path, index=False)  


# Bidzan 2012

In [130]:
file_path = "../Desktop/Training/bidzan2012.pdf"

# Extract content from PDF
bidzan_2012_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_bidzan_2012 = query_gpt4_text(bidzan_2012_text)

In [131]:
print("Extracted Data:", structured_data_bidzan_2012.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Bidzan et al., 2012",
    "Full Population Sample Size": "301",
    "Intervention": "Vortioxetine: 5 mg/day",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "68.7",
    "Mean HAMA Score": "26.3",
    "Mean Population Age (Year)": "45.2",
    "Attrition Rate (%)": "NA",
    "Full Sponsor Name": "Takeda Pharmaceutical Company, Ltd.",
    "Follow-up Time (Weeks)": "8",
    "Diagnostic Criteria": "DSM-IV-TR"
}
{
    "Last Name of Main Author and Year": "Bidzan et al., 2012",
    "Full Population Sample Size": "301",
    "Intervention": "Placebo",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "61.6",
    "Mean HAMA Score": "26.8",
    "Mean Population Age (Year)": "45.3",
    "Attrition Rate (%)": "NA",
    "Full Sponsor Name": "Takeda Pharmaceutical Company, Ltd.",
    "Follow-up Time (Weeks)": "8",
    "Diagnostic Criteria": "DSM-IV-TR"
}
```


In [135]:
#Let's make this a method
def json_parsing(output):
    text = output.choices[0].message.content
    text = text.replace("```json\n{\n", "```json\n[\n    {\n")
    text = text.replace("\n}\n```", "\n    }\n]\n```")
    text = text.replace("}\n{","},\n{")
    
    if text.startswith("```json"):
        text = text[len("```json"):].strip()
    if text.endswith("```"):
        text = text[:-len("```")].strip()
    
    # Step 2: Parse the JSON list directly
    try:
        data = json.loads(text)  # Parse the JSON list directly
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        data = []
    
    # Step 3: Convert the list of dictionaries into a pandas DataFrame
    return pd.DataFrame(data)


In [156]:
bidzan_2012_df = json_parsing(structured_data_bidzan_2012)

In [157]:
bidzan_2012_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Bidzan et al., 2012",301,Vortioxetine: 5 mg/day,White,68.7,26.3,45.2,NA,"Takeda Pharmaceutical Company, Ltd.",8,DSM-IV-TR
1,"Bidzan et al., 2012",301,Placebo,White,61.6,26.8,45.3,NA,"Takeda Pharmaceutical Company, Ltd.",8,DSM-IV-TR


In [145]:
df_truth[df_truth["References"] == "Bidzan et al., 2012"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
59,"Bidzan et al., 2012",301,vortioxetine: 5 mg/day,White,68.7,26.3,45.0,14.7,Takeda,8.0,dsm-iv
60,"Bidzan et al., 2012",301,placebo,White,61.6,26.8,45.3,16.6,Takeda,8.0,dsm-iv


In [158]:
bidzan_2012_df = rename_columns(bidzan_2012_df)
bidzan_2012_df = preprocess_interventions_and_diag(bidzan_2012_df)
bidzan_2012_df = preprocess_numerical_columns(bidzan_2012_df, numerical_cols)

In [159]:
bidzan_2012_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Bidzan et al., 2012",301,vortioxetine: 5 mg/day,White,68.7,26.3,45.2,NA,"Takeda Pharmaceutical Company, Ltd.",8,dsm-iv-tr
1,"Bidzan et al., 2012",301,placebo,White,61.6,26.8,45.3,NA,"Takeda Pharmaceutical Company, Ltd.",8,dsm-iv-tr


In [173]:
bidzan2012_results = testing(df_truth, bidzan_2012_df)

In [174]:
bidzan2012_results

,References,Interventions,Main race,Sponsor,Diagnosis criteria,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bidzan et al., 2012",placebo,True,False,True,True,True,True,True,False,True
1,"Bidzan et al., 2012",vortioxetine: 5 mg/day,True,False,True,True,True,True,True,False,True


In [175]:
bidzan2012_results_final = ensure_all_truth_interventions_present(bidzan2012_results,df_truth)

In [176]:
bidzan2012_results_final

,References,Interventions,Main race,Sponsor,Diagnosis criteria,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bidzan et al., 2012",placebo,True,False,True,True,True,True,True,False,True
1,"Bidzan et al., 2012",vortioxetine: 5 mg/day,True,False,True,True,True,True,True,False,True


In [177]:
file_path = os.path.join("data_text", "bidzan2012data.csv")  
bidzan_2012_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "bidzan2012results.csv")  
bidzan2012_results_final.to_csv(file_path, index=False)  


## Coric 2010

In [178]:
file_path = "../Desktop/Training/coric2010.pdf"

# Extract content from PDF
coric_2010_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_coric_2010 = query_gpt4_text(coric_2010_text)

In [180]:
print("Extracted Data:", structured_data_coric_2010.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Coric et al., 2010",
        "Full Population Sample Size": "260",
        "Intervention": "Pexacerfont: 100 mg/day",
        "Main Race": "White",
        "Percent of Study Population that is Female (%)": "100",
        "Mean HAMA Score": "24.0",
        "Mean Population Age (Year)": "38.5",
        "Attrition Rate (%)": "5",
        "Full Sponsor Name": "Bristol-Myers Squibb Company",
        "Follow-up Time (Weeks)": "8",
        "Diagnostic Criteria": "DSM-IV-TR"
    },
    {
        "Last Name of Main Author and Year": "Coric et al., 2010",
        "Full Population Sample Size": "260",
        "Intervention": "Placebo",
        "Main Race": "White",
        "Percent of Study Population that is Female (%)": "100",
        "Mean HAMA Score": "24.7",
        "Mean Population Age (Year)": "39.3",
        "Attrition Rate (%)": "3",
        "Full Sponsor Name": "Bristol-Myers Squibb Company",
        "Follow-u

In [181]:
coric_2010_df = json_parsing(structured_data_coric_2010)

In [182]:
coric_2010_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Coric et al., 2010",260,Pexacerfont: 100 mg/day,White,100,24.0,38.5,5,Bristol-Myers Squibb Company,8,DSM-IV-TR
1,"Coric et al., 2010",260,Placebo,White,100,24.7,39.3,3,Bristol-Myers Squibb Company,8,DSM-IV-TR
2,"Coric et al., 2010",260,Escitalopram: 20 mg/day,White,100,23.5,38.8,10,Bristol-Myers Squibb Company,8,DSM-IV-TR


In [184]:
df_truth[df_truth["References"] == "Coric et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
72,"Coric et al., 2010",157,escitalopram: 20 mg/day,White,100.0,23.5,38.8,20.8,Bristol-Myers,8.0,dsm-iv
73,"Coric et al., 2010",157,placebo,White,100.0,24.7,39.3,23.1,Bristol-Myers,8.0,dsm-iv


# Important Notice: ChatGPT has identified a treatment that was omitted in the orignal study for unknown reason: Pexacerfont. I will update df_truth by manually reading the Coric 2010 study and reload it here:

In [185]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [210]:
df_truth = preprocess_interventions_and_diag(df_truth)
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)

In [215]:
coric_2010_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Coric et al., 2010",260,pexacerfont: 100 mg/day,White,100,24.0,38.5,5,Bristol-Myers Squibb Company,8,dsm-iv-tr
1,"Coric et al., 2010",260,placebo,White,100,24.7,39.3,3,Bristol-Myers Squibb Company,8,dsm-iv-tr
2,"Coric et al., 2010",260,escitalopram: 20 mg/day,White,100,23.5,38.8,10,Bristol-Myers Squibb Company,8,dsm-iv-tr


In [211]:
df_truth[df_truth["References"] == "Coric et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
72,"Coric et al., 2010",260,pexacerfont: 100 mg/day,White,100.0,24.0,38.5,26.2,Bristol-Myers,8.0,dsm-iv
73,"Coric et al., 2010",260,escitalopram: 20 mg/day,White,100.0,23.5,38.8,20.8,Bristol-Myers,8.0,dsm-iv
74,"Coric et al., 2010",260,placebo,White,100.0,24.7,39.3,23.1,Bristol-Myers,8.0,dsm-iv


In [194]:
coric_2010_df = rename_columns(coric_2010_df)
coric_2010_df = preprocess_interventions_and_diag(coric_2010_df)
coric_2010_df = preprocess_numerical_columns(coric_2010_df, numerical_cols)

In [221]:
coric2010_results = testing(df_truth, coric_2010_df)

In [222]:
coric2010_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Coric et al., 2010",escitalopram: 20 mg/day,True,True,True,True,True,True,True,False,True
1,"Coric et al., 2010",placebo,True,True,True,True,True,True,True,False,True
2,"Coric et al., 2010",pexacerfont: 100 mg/day,True,True,True,True,True,True,True,False,True


In [223]:
coric2010_results_final = ensure_all_truth_interventions_present(coric2010_results,df_truth)

In [224]:
coric2010_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Coric et al., 2010",escitalopram: 20 mg/day,True,True,True,True,True,True,True,False,True
1,"Coric et al., 2010",placebo,True,True,True,True,True,True,True,False,True
2,"Coric et al., 2010",pexacerfont: 100 mg/day,True,True,True,True,True,True,True,False,True


In [225]:
file_path = os.path.join("data_text", "coric2010data.csv")  
coric_2010_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "coric2010results.csv")  
coric2010_results_final.to_csv(file_path, index=False)  


# Stein 2014

In [242]:
file_path = "../Desktop/Training/stein2014.pdf"

# Extract content from PDF
stein_2014_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_stein_2014 = query_gpt4_text(stein_2014_text)

In [243]:
print("Extracted Data:", structured_data_stein_2014.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Stein et al., 2014",
    "Full Population Sample Size": "412",
    "Intervention": "Agomelatine: 25–50 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "71.6",
    "Mean HAMA Score": "28.6",
    "Mean Population Age (Year)": "42.6",
    "Attrition Rate (%)": "16.5",
    "Full Sponsor Name": "Servier",
    "Follow-up Time (Weeks)": "12",
    "Diagnostic Criteria": "DSM-IV-TR"
},
{
    "Last Name of Main Author and Year": "Stein et al., 2014",
    "Full Population Sample Size": "412",
    "Intervention": "Escitalopram: 10–20 mg/day",
    "Main Race": "NA",
    "Percent of Study Population that is Female (%)": "71.6",
    "Mean HAMA Score": "28.6",
    "Mean Population Age (Year)": "42.6",
    "Attrition Rate (%)": "18.3",
    "Full Sponsor Name": "Servier",
    "Follow-up Time (Weeks)": "12",
    "Diagnostic Criteria": "DSM-IV-TR"
},
{
    "Last Name of Main Author and Year": "Stein e

In [244]:
stein_2014_df = json_parsing(structured_data_stein_2014)

In [249]:
stein_2014_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Stein et al., 2014",412,agomelatine: 25-50 mg/day,NA,71.6,28.6,42.6,16.5,Servier,12,dsm-iv-tr
1,"Stein et al., 2014",412,escitalopram: 10-20 mg/day,NA,71.6,28.6,42.6,18.3,Servier,12,dsm-iv-tr
2,"Stein et al., 2014",412,placebo,NA,71.6,28.2,42.6,26.7,Servier,12,dsm-iv-tr


In [250]:
df_truth[df_truth["References"] == "Stein et al., 2014"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
14,"Stein et al., 2014",412,agomelatine: 25-50 mg/day,NA,74.8,28.6,43.6,16.5,Servier,12.0,dsm-iv
15,"Stein et al., 2014",412,escitalopram: 10-20 mg/day,NA,68.3,28.6,41.2,26.7,Servier,12.0,dsm-iv
16,"Stein et al., 2014",412,placebo,NA,71.8,28.2,43.0,17.6,Servier,12.0,dsm-iv


In [247]:
stein_2014_df = rename_columns(stein_2014_df)
stein_2014_df = preprocess_interventions_and_diag(stein_2014_df)
stein_2014_df = preprocess_numerical_columns(stein_2014_df, numerical_cols)

In [248]:
stein2014_results = testing(df_truth, stein_2014_df)

# Note that on first run, ChatGPT returned interventions in the format "escitalopram: 10-20 mg/dy" which broke the matching logic. Upon re-running it returned the correct format. This will be noted for the prompt in the Testing data

In [251]:
stein2014_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2014",agomelatine: 25-50 mg/day,True,True,True,True,False,True,False,True,True
1,"Stein et al., 2014",placebo,True,True,True,True,True,True,True,False,True
2,"Stein et al., 2014",escitalopram: 10-20 mg/day,True,True,True,True,False,True,False,False,True


In [252]:
stein2014_results_final = ensure_all_truth_interventions_present(stein2014_results,df_truth)

In [253]:
stein2014_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Stein et al., 2014",agomelatine: 25-50 mg/day,True,True,True,True,False,True,False,True,True
1,"Stein et al., 2014",placebo,True,True,True,True,True,True,True,False,True
2,"Stein et al., 2014",escitalopram: 10-20 mg/day,True,True,True,True,False,True,False,False,True


In [254]:
file_path = os.path.join("data_text", "stein2014data.csv")  
stein_2014_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "stein2014results.csv")  
stein2014_results_final.to_csv(file_path, index=False)  


# Feltner 2003

In [264]:
file_path = "../Desktop/Training/feltner2003.pdf"

# Extract content from PDF
feltner_2003_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_feltner_2003 = query_gpt4_text(feltner_2003_text)

In [265]:
print("Extracted Data:", structured_data_feltner_2003.choices[0].message.content)

Extracted Data: ```json
[
  {
    "Last Name of Main Author and Year": "Feltner et al., 2003",
    "Full Population Sample Size": "271",
    "Intervention": "Pregabalin: 150 mg/day",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "51.4",
    "Mean HAMA Score": "24.9",
    "Mean Population Age (Year)": "37.9",
    "Attrition Rate (%)": "24.3",
    "Full Sponsor Name": "Pfizer Global Research and Development",
    "Follow-up Time (Weeks)": "4",
    "Diagnostic Criteria": "DSM-IV"
  },
  {
    "Last Name of Main Author and Year": "Feltner et al., 2003",
    "Full Population Sample Size": "271",
    "Intervention": "Pregabalin: 600 mg/day",
    "Main Race": "White",
    "Percent of Study Population that is Female (%)": "50.0",
    "Mean HAMA Score": "25.4",
    "Mean Population Age (Year)": "36.3",
    "Attrition Rate (%)": "30.3",
    "Full Sponsor Name": "Pfizer Global Research and Development",
    "Follow-up Time (Weeks)": "4",
    "Diagnostic Criteria"

In [267]:
feltner_2003_df = json_parsing(structured_data_feltner_2003)

In [268]:
feltner_2003_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Study Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Feltner et al., 2003",271,Pregabalin: 150 mg/day,White,51.4,24.9,37.9,24.3,Pfizer Global Research and Development,4,DSM-IV
1,"Feltner et al., 2003",271,Pregabalin: 600 mg/day,White,50.0,25.4,36.3,30.3,Pfizer Global Research and Development,4,DSM-IV
2,"Feltner et al., 2003",271,Lorazepam: 6 mg/day,White,58.8,24.7,39.2,47.1,Pfizer Global Research and Development,4,DSM-IV
3,"Feltner et al., 2003",271,Placebo,White,50.7,24.8,37.8,28.4,Pfizer Global Research and Development,4,DSM-IV


In [259]:
df_truth[df_truth["References"] == "Feltner et al., 2003"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
11,"Feltner et al., 2003",271,pregabalin: 600 mg/day,White,50.0,25.4,36.3,30.3,Pfizer,4.0,dsm-iv
12,"Feltner et al., 2003",271,lorazepam: 6 mg/day,White,58.8,24.7,39.2,47.1,Pfizer,4.0,dsm-iv
13,"Feltner et al., 2003",271,placebo,White,50.7,24.8,37.8,28.4,Pfizer,4.0,dsm-iv


# Note: ChatGPT identified a treatment omitted in the meta-analysis study. Pregablin: 150 mg/day. So I put it in the truth excel and reloaded the file

In [260]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [290]:
df_truth = preprocess_interventions_and_diag(df_truth)
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)

In [262]:
df_truth[df_truth["References"] == "Feltner et al., 2003"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
11,"Feltner et al., 2003",271,pregabalin: 150 mg/day,White,51.4,24.9,37.9,24.3,Pfizer,4.0,dsm-iv
12,"Feltner et al., 2003",271,pregabalin: 600 mg/day,White,50.0,25.4,36.3,30.3,Pfizer,4.0,dsm-iv
13,"Feltner et al., 2003",271,lorazepam: 6 mg/day,White,58.8,24.7,39.2,47.1,Pfizer,4.0,dsm-iv
14,"Feltner et al., 2003",271,placebo,White,50.7,24.8,37.8,28.4,Pfizer,4.0,dsm-iv


In [269]:
feltner_2003_df = rename_columns(feltner_2003_df)
feltner_2003_df = preprocess_interventions_and_diag(feltner_2003_df)
feltner_2003_df = preprocess_numerical_columns(feltner_2003_df, numerical_cols)

In [270]:
feltner2003_results = testing(df_truth, feltner_2003_df)

In [271]:
feltner2003_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Feltner et al., 2003",pregabalin: 600 mg/day,True,True,True,True,True,True,True,True,True
1,"Feltner et al., 2003",pregabalin: 150 mg/day,True,True,True,True,True,True,True,True,True
2,"Feltner et al., 2003",placebo,True,True,True,True,True,True,True,True,True
3,"Feltner et al., 2003",lorazepam: 6 mg/day,True,True,True,True,True,True,True,True,True


In [272]:
feltner2003_results_final = ensure_all_truth_interventions_present(feltner2003_results,df_truth)

In [274]:
feltner2003_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Feltner et al., 2003",pregabalin: 600 mg/day,True,True,True,True,True,True,True,True,True
1,"Feltner et al., 2003",pregabalin: 150 mg/day,True,True,True,True,True,True,True,True,True
2,"Feltner et al., 2003",placebo,True,True,True,True,True,True,True,True,True
3,"Feltner et al., 2003",lorazepam: 6 mg/day,True,True,True,True,True,True,True,True,True


In [275]:
file_path = os.path.join("data_text", "feltner2003data.csv")  
feltner_2003_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "feltner2003results.csv")  
feltner2003_results_final.to_csv(file_path, index=False)  


# Koponen 2007

In [297]:
file_path = "../Desktop/Training/koponen2007.pdf"

# Extract content from PDF
koponen_2007_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_koponen_2007 = query_gpt4_text(koponen_2007_text)

In [298]:
print("Extracted Data:", structured_data_koponen_2007.choices[0].message.content)

Extracted Data: ```json
{
    "Last Name of Main Author and Year": "Koponen et al., 2007",
    "Full Population Sample Size": "513",
    "Intervention": "Duloxetine: 60 mg/day",
    "Main Race": "Caucasian",
    "Percent of Study Population that is Female (%)": "67.8",
    "Mean HAMA Score": "25.0",
    "Mean Population Age (Year)": "43.8",
    "Attrition Rate (%)": "24.2",
    "Full Sponsor Name": "Eli Lilly and Co. and Boehringer Ingelheim",
    "Follow-up Time (Weeks)": "9",
    "Diagnostic Criteria": "DSM-IV"
},
{
    "Last Name of Main Author and Year": "Koponen et al., 2007",
    "Full Population Sample Size": "513",
    "Intervention": "Duloxetine: 120 mg/day",
    "Main Race": "Caucasian",
    "Percent of Study Population that is Female (%)": "67.8",
    "Mean HAMA Score": "25.2",
    "Mean Population Age (Year)": "43.8",
    "Attrition Rate (%)": "24.2",
    "Full Sponsor Name": "Eli Lilly and Co. and Boehringer Ingelheim",
    "Follow-up Time (Weeks)": "9",
    "Diagnostic Cr

In [299]:
koponen_2007_df = json_parsing(structured_data_koponen_2007)

In [289]:
koponen_2007_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Koponen et al., 2007",513,duloxetine: 60 mg/day,White,67.8,25.0,43.8,24.2,Eli Lilly and Co. and Boehringer Ingelheim,9,dsm-iv
1,"Koponen et al., 2007",513,duloxetine: 120 mg/day,White,67.8,25.2,43.8,24.2,Eli Lilly and Co. and Boehringer Ingelheim,9,dsm-iv
2,"Koponen et al., 2007",513,placebo,White,67.8,25.8,43.8,24.2,Eli Lilly and Co. and Boehringer Ingelheim,9,dsm-iv


In [300]:
df_truth[df_truth["References"] == "Koponen et al., 2007"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
64,"Koponen et al., 2007",513,duloxetine: 60 mg/day,Caucasian,64.3,25.0,43.1,NaN,Eli Lilly,9.0,dsm-iv
65,"Koponen et al., 2007",513,duloxetine: 120 mg/day,Caucasian,72.3,25.2,44.1,NaN,Eli Lilly,9.0,dsm-iv
66,"Koponen et al., 2007",513,placebo,Caucasian,66.9,25.8,44.1,NaN,Eli Lilly,9.0,dsm-iv


In [286]:
koponen_2007_df = rename_columns(koponen_2007_df)
koponen_2007_df = preprocess_interventions_and_diag(koponen_2007_df)
koponen_2007_df = preprocess_numerical_columns(koponen_2007_df, numerical_cols)

In [292]:
koponen2007_results = testing(df_truth, koponen_2007_df)

In [293]:
koponen2007_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Koponen et al., 2007",duloxetine: 60 mg/day,False,True,True,True,False,True,False,False,True
1,"Koponen et al., 2007",placebo,False,True,True,True,False,True,True,False,True
2,"Koponen et al., 2007",duloxetine: 120 mg/day,False,True,True,True,False,True,True,False,True


# Note: ChatGPT appears to have possibly converted Caucasian to White, and seemed to group female % and attrition rate % instead of differentiating for each study.

In [301]:
koponen2007_results_final = ensure_all_truth_interventions_present(koponen2007_results,df_truth)

In [302]:
koponen2007_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Koponen et al., 2007",duloxetine: 60 mg/day,False,True,True,True,False,True,False,False,True
1,"Koponen et al., 2007",placebo,False,True,True,True,False,True,True,False,True
2,"Koponen et al., 2007",duloxetine: 120 mg/day,False,True,True,True,False,True,True,False,True


In [303]:
file_path = os.path.join("data_text", "koponen2007data.csv")  
koponen_2007_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "koponen2007results.csv")  
koponen2007_results_final.to_csv(file_path, index=False)  


# Montgomery 2008

In [317]:
file_path = "../Desktop/Training/montgomery2008.pdf"

# Extract content from PDF
montgomery_2008_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_montgomery_2008 = query_gpt4_text(montgomery_2008_text)

In [318]:
print("Extracted Data:", structured_data_montgomery_2008.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Montgomery et al., 2008",
        "Full Population Sample Size": "273",
        "Intervention": "Pregabalin: 150-600 mg/day",
        "Main Race": "White",
        "Percent of Intervention Population that is Female (%)": "79",
        "Mean HAMA Score": "26",
        "Mean Population Age (Year)": "72.4",
        "Attrition Rate (%)": "24.9",
        "Full Sponsor Name": "Pfizer Inc.",
        "Follow-up Time (Weeks)": "8",
        "Diagnostic Criteria": "DSM-IV"
    },
    {
        "Last Name of Main Author and Year": "Montgomery et al., 2008",
        "Full Population Sample Size": "273",
        "Intervention": "Placebo",
        "Main Race": "White",
        "Percent of Intervention Population that is Female (%)": "75",
        "Mean HAMA Score": "26",
        "Mean Population Age (Year)": "72.2",
        "Attrition Rate (%)": "28.1",
        "Full Sponsor Name": "Pfizer Inc.",
        "Follow-up Time (We

In [327]:
montgomery_2008_df = json_parsing(structured_data_montgomery_2008)

In [338]:
montgomery_2008_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Montgomery et al., 2008",273,pregabalin: 150-600 mg/day,White,79,26,72.4,24.9,Pfizer Inc.,8,dsm-iv
1,"Montgomery et al., 2008",273,placebo,White,75,26,72.2,28.1,Pfizer Inc.,8,dsm-iv


In [339]:
df_truth[df_truth["References"] == "Montgomery et al., 2008"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
26,"Montgomery et al., 2008",273,pregabalin: 150-600 mg/day,White,79.0,27.0,72.4,24.9,Pfizer,8.0,dsm-iv
27,"Montgomery et al., 2008",273,placebo,White,75.0,26.0,72.2,28.0,Pfizer,8.0,dsm-iv


In [330]:
montgomery_2008_df = rename_columns(montgomery_2008_df)
montgomery_2008_df = preprocess_interventions_and_diag(montgomery_2008_df)
montgomery_2008_df = preprocess_numerical_columns(montgomery_2008_df, numerical_cols)

In [326]:
montgomery2008_results = testing(df_truth, montgomery_2008_df)

In [333]:
montgomery2008_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Montgomery et al., 2008",placebo,True,True,True,True,True,True,True,True,True
1,"Montgomery et al., 2008",pregabalin: 150-600 mg/day,True,True,True,True,True,False,True,True,True


# Note: ChatGPT has the correct effective dosage tested, 150-600 mg/day. The treatment was started at 50 mg yes, but then was upped to 150 mg/day over 5 days, likely for tolarability. I will change the truth excel and reloaded. 

In [313]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)
df_truth = preprocess_interventions_and_diag(df_truth)
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)

In [334]:
montgomery2008_results = testing(df_truth, montgomery_2008_df)

In [335]:
montgomery2008_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Montgomery et al., 2008",placebo,True,True,True,True,True,True,True,True,True
1,"Montgomery et al., 2008",pregabalin: 150-600 mg/day,True,True,True,True,True,False,True,True,True


In [336]:
montgomery2008_results_final = ensure_all_truth_interventions_present(montgomery2008_results,df_truth)

In [337]:
montgomery2008_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Montgomery et al., 2008",placebo,True,True,True,True,True,True,True,True,True
1,"Montgomery et al., 2008",pregabalin: 150-600 mg/day,True,True,True,True,True,False,True,True,True


In [341]:
file_path = os.path.join("data_text", "montgomery2008data.csv")  
montgomery_2008_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "montgomery2008results.csv")  
montgomery2008_results_final.to_csv(file_path, index=False)  

# Pollock 2008b

In [342]:
file_path = "../Desktop/Training/pollock2008b.pdf"

# Extract content from PDF
pollock_2008b_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_pollock_2008b = query_gpt4_text(pollock_2008b_text)

In [344]:
print("Extracted Data:", structured_data_pollock_2008b.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Pollack et al., 2008",
        "Full Population Sample Size": "1790",
        "Intervention": "Tiagabine: 4 mg/day",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "59.5",
        "Mean HAMA Score": "26.5",
        "Mean Population Age (Year)": "40.8",
        "Attrition Rate (%)": "36",
        "Full Sponsor Name": "Cephalon, Inc.",
        "Follow-up Time (Weeks)": "10",
        "Diagnostic Criteria": "DSM-IV"
    },
    {
        "Last Name of Main Author and Year": "Pollack et al., 2008",
        "Full Population Sample Size": "1790",
        "Intervention": "Tiagabine: 8 mg/day",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "59.5",
        "Mean HAMA Score": "27.3",
        "Mean Population Age (Year)": "39.4",
        "Attrition Rate (%)": "41",
        "Full Sponsor Name": "Cephalon, Inc.",
        "Follow-up Ti

# Note: Analyzing Pollock 2008 manually, I found that this study is a meta analysis of 3 independant, seperate trials. For this reason, I will omit it from the Testing and Training folders, as including it in accuracy calculations would be incorrect, as techincally this is not a clinical trial report, but rather a meta analysis itself.

# Bandelow 2010

In [345]:
file_path = "../Desktop/Training/bandelow2010.pdf"

# Extract content from PDF
bandelow_2010_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_bandelow_2010 = query_gpt4_text(bandelow_2010_text)

In [346]:
print("Extracted Data:", structured_data_bandelow_2010.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Bandelow et al., 2010",
        "Full Population Sample Size": "873",
        "Intervention": "Placebo",
        "Main Race": "White",
        "Percent of Intervention Population that is Female (%)": "62.2",
        "Mean HAMA Score": "27.3",
        "Mean Population Age (Year)": "41.2",
        "Attrition Rate (%)": "18.9",
        "Full Sponsor Name": "AstraZeneca Pharmaceuticals",
        "Follow-up Time (Weeks)": "10",
        "Diagnostic Criteria": "DSM-IV"
    },
    {
        "Last Name of Main Author and Year": "Bandelow et al., 2010",
        "Full Population Sample Size": "873",
        "Intervention": "Quetiapine XR: 50 mg/day",
        "Main Race": "White",
        "Percent of Intervention Population that is Female (%)": "68.0",
        "Mean HAMA Score": "26.9",
        "Mean Population Age (Year)": "40.7",
        "Attrition Rate (%)": "25.8",
        "Full Sponsor Name": "AstraZeneca Pharmaceut

In [347]:
bandelow_2010_df = json_parsing(structured_data_bandelow_2010)

In [348]:
bandelow_2010_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Intervention Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Bandelow et al., 2010",873,Placebo,White,62.2,27.3,41.2,18.9,AstraZeneca Pharmaceuticals,10,DSM-IV
1,"Bandelow et al., 2010",873,Quetiapine XR: 50 mg/day,White,68.0,26.9,40.7,25.8,AstraZeneca Pharmaceuticals,10,DSM-IV
2,"Bandelow et al., 2010",873,Quetiapine XR: 150 mg/day,White,66.7,26.6,42.3,25.2,AstraZeneca Pharmaceuticals,10,DSM-IV
3,"Bandelow et al., 2010",873,Paroxetine: 20 mg/day,White,64.5,27.1,41.6,20.3,AstraZeneca Pharmaceuticals,10,DSM-IV


In [350]:
bandelow_2010_df.Intervention[1]

'Quetiapine XR: 50 mg/day'

In [349]:
df_truth[df_truth["References"] == "Bandelow et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
45,"Bandelow et al., 2010",873,quetiapine: 50 mg/day,White,68.0,26.9,40.7,25.8,AstraZeneca,8.0,dsm-iv
46,"Bandelow et al., 2010",873,quetiapine: 150 mg/day,White,66.7,26.6,42.3,25.2,AstraZeneca,8.0,dsm-iv
47,"Bandelow et al., 2010",873,paroxetine: 20 mg/day,White,64.5,27.1,41.6,20.3,AstraZeneca,8.0,dsm-iv
48,"Bandelow et al., 2010",873,placebo,White,62.2,27.3,41.2,18.9,AstraZeneca,8.0,dsm-iv


# Note: ChatGPT specified the correct formulation of Quetiapine (XR), while the truth excel only specifies the drug. For this reason I will update the truth excel to have the correct drug.

In [359]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)
df_truth = preprocess_interventions_and_diag(df_truth)
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)

In [364]:
bandelow_2010_df

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
0,"Bandelow et al., 2010",873,placebo,White,62.2,27.3,41.2,18.9,AstraZeneca Pharmaceuticals,10,dsm-iv
1,"Bandelow et al., 2010",873,quetiapine xr: 50 mg/day,White,68.0,26.9,40.7,25.8,AstraZeneca Pharmaceuticals,10,dsm-iv
2,"Bandelow et al., 2010",873,quetiapine xr: 150 mg/day,White,66.7,26.6,42.3,25.2,AstraZeneca Pharmaceuticals,10,dsm-iv
3,"Bandelow et al., 2010",873,paroxetine: 20 mg/day,White,64.5,27.1,41.6,20.3,AstraZeneca Pharmaceuticals,10,dsm-iv


In [360]:
df_truth[df_truth["References"] == "Bandelow et al., 2010"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
45,"Bandelow et al., 2010",873,quetiapine xr: 50 mg/day,White,68.0,26.9,40.7,25.8,AstraZeneca,8.0,dsm-iv
46,"Bandelow et al., 2010",873,quetiapine xr: 150 mg/day,White,66.7,26.6,42.3,25.2,AstraZeneca,8.0,dsm-iv
47,"Bandelow et al., 2010",873,paroxetine: 20 mg/day,White,64.5,27.1,41.6,20.3,AstraZeneca,8.0,dsm-iv
48,"Bandelow et al., 2010",873,placebo,White,62.2,27.3,41.2,18.9,AstraZeneca,8.0,dsm-iv


In [361]:
bandelow_2010_df = rename_columns(bandelow_2010_df)
bandelow_2010_df = preprocess_interventions_and_diag(bandelow_2010_df)
bandelow_2010_df = preprocess_numerical_columns(bandelow_2010_df, numerical_cols)

In [362]:
bandelow2010_results = testing(df_truth, bandelow_2010_df)

In [363]:
bandelow2010_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bandelow et al., 2010",paroxetine: 20 mg/day,True,True,True,True,True,True,True,True,False
1,"Bandelow et al., 2010",quetiapine xr: 50 mg/day,True,True,True,True,True,True,True,True,False
2,"Bandelow et al., 2010",placebo,True,True,True,True,True,True,True,True,False
3,"Bandelow et al., 2010",quetiapine xr: 150 mg/day,True,True,True,True,True,True,True,True,False


# Note: Failed Follow-up time due to the study technically lasting 10 weeks, with results recorded at week 8, and 2 weeks of drug discontinuation

In [366]:
bandelow2010_results_final = ensure_all_truth_interventions_present(bandelow2010_results,df_truth)

In [367]:
bandelow2010_results_final

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Bandelow et al., 2010",paroxetine: 20 mg/day,True,True,True,True,True,True,True,True,False
1,"Bandelow et al., 2010",quetiapine xr: 50 mg/day,True,True,True,True,True,True,True,True,False
2,"Bandelow et al., 2010",placebo,True,True,True,True,True,True,True,True,False
3,"Bandelow et al., 2010",quetiapine xr: 150 mg/day,True,True,True,True,True,True,True,True,False


In [368]:
file_path = os.path.join("data_text", "bandelow2010data.csv")  
bandelow_2010_df.to_csv(file_path, index=False)  

file_path = os.path.join("results_text", "bandelow2010results.csv")  
bandelow2010_results_final.to_csv(file_path, index=False)  

# Mezhebovsky 2013

In [369]:
file_path = "../Desktop/Training/mezhebovsky2013.pdf"

# Extract content from PDF
mezhebovsky_2013_text = extract_pdf_text(file_path)

# Query GPT-4
structured_data_mezhebovsky_2013 = query_gpt4_text(mezhebovsky_2013_text)

In [370]:
print("Extracted Data:", structured_data_mezhebovsky_2013.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Mezhebovsky et al., 2012",
        "Full Population Sample Size": "450",
        "Intervention": "Quetiapine XR: 50-300 mg/day",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "NA",
        "Mean HAMA Score": "NA",
        "Mean Population Age (Year)": "NA",
        "Attrition Rate (%)": "NA",
        "Full Sponsor Name": "AstraZeneca Pharmaceuticals",
        "Follow-up Time (Weeks)": "11",
        "Diagnostic Criteria": "DSM-IV"
    },
    {
        "Last Name of Main Author and Year": "Mezhebovsky et al., 2012",
        "Full Population Sample Size": "450",
        "Intervention": "Placebo",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "NA",
        "Mean HAMA Score": "NA",
        "Mean Population Age (Year)": "NA",
        "Attrition Rate (%)": "NA",
        "Full Sponsor Name": "AstraZeneca Pharmaceuticals",
    

# Note: ChatGPT specified the correct formulation of Quetiapine (XR), while the truth excel only specifies the drug. For this reason I will update the truth excel to have the correct drug. Additionally, while the original study was indicated as published in 2013, manual review of the study appears to show it published in 2012, which ChatGPT correctly identified. I edit this in the truth excel as well. 

In [375]:
from openpyxl import load_workbook

file_path = '../Downloads/all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)
df_truth = preprocess_interventions_and_diag(df_truth)
df_truth = preprocess_numerical_columns(df_truth, numerical_cols)

In [382]:
print("Extracted Data:", structured_data_mezhebovsky_2013.choices[0].message.content)

Extracted Data: ```json
[
    {
        "Last Name of Main Author and Year": "Mezhebovsky et al., 2012",
        "Full Population Sample Size": "450",
        "Intervention": "Quetiapine XR: 50-300 mg/day",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "NA",
        "Mean HAMA Score": "NA",
        "Mean Population Age (Year)": "NA",
        "Attrition Rate (%)": "NA",
        "Full Sponsor Name": "AstraZeneca Pharmaceuticals",
        "Follow-up Time (Weeks)": "11",
        "Diagnostic Criteria": "DSM-IV"
    },
    {
        "Last Name of Main Author and Year": "Mezhebovsky et al., 2012",
        "Full Population Sample Size": "450",
        "Intervention": "Placebo",
        "Main Race": "NA",
        "Percent of Intervention Population that is Female (%)": "NA",
        "Mean HAMA Score": "NA",
        "Mean Population Age (Year)": "NA",
        "Attrition Rate (%)": "NA",
        "Full Sponsor Name": "AstraZeneca Pharmaceuticals",
    

In [376]:
mezhebovsky_2013_df = json_parsing(structured_data_mezhebovsky_2013)

In [377]:
mezhebovsky_2013_df

,Last Name of Main Author and Year,Full Population Sample Size,Intervention,Main Race,Percent of Intervention Population that is Female (%),Mean HAMA Score,Mean Population Age (Year),Attrition Rate (%),Full Sponsor Name,Follow-up Time (Weeks),Diagnostic Criteria
0,"Mezhebovsky et al., 2012",450,Quetiapine XR: 50-300 mg/day,NA,NA,NA,NA,NA,AstraZeneca Pharmaceuticals,11,DSM-IV
1,"Mezhebovsky et al., 2012",450,Placebo,NA,NA,NA,NA,NA,AstraZeneca Pharmaceuticals,11,DSM-IV


In [378]:
df_truth[df_truth["References"] == "Mezhebovsky et al., 2012"]

,References,Sample size,Interventions,Main race,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Sponsor,Follow-up time (weeks),Diagnosis criteria
62,"Mezhebovsky et al., 2012",450,quetiapine xr: 50-300 mg/day,White,72.1,25.2,70.3,20.2,AstraZeneca,9.0,dsm-iv
63,"Mezhebovsky et al., 2012",450,placebo,White,69.0,25.1,70.6,26.0,AstraZeneca,9.0,dsm-iv


In [379]:
mezhebovsky_2013_df = rename_columns(mezhebovsky_2013_df)
mezhebovsky_2013_df = preprocess_interventions_and_diag(mezhebovsky_2013_df)
mezhebovsky_2013_df = preprocess_numerical_columns(mezhebovsky_2013_df, numerical_cols)

In [380]:
mezhebovsky2013_results = testing(df_truth, mezhebovsky_2013_df)

In [381]:
mezhebovsky2013_results

,References,Interventions,Main race,Diagnosis criteria,Sponsor,Sample size,Female (%),Mean HAMA,Mean age (Year),Attrition rate (%),Follow-up time (weeks)
0,"Mezhebovsky et al., 2012",quetiapine xr: 50-300 mg/day,False,True,True,True,False,False,False,False,False
1,"Mezhebovsky et al., 2012",placebo,False,True,True,True,False,False,False,False,False


# Upon manually reviewing and attempting to confirm the results missed by ChatGPT, I was unable to find the results in the truth excel in the actual study document. For this reason I will be excluding this study from the training set.